In [2]:
import pandas as pd
import random 
import os

pd.set_option('display.max_rows', 50)


from functions import sharpe_ratio_calculation, generate_rand_portfolios, select_top_five, join_stocks_crypto, test_for_silhouette_score, run_clustering_model, label_balance

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


ImportError: cannot import name 'test_for_silhouette_score' from 'functions' (c:\Users\Kirill\Documents\Projects\thesis\functions.py)

In [ ]:
df_all_stocks = pd.read_csv('stocks_data_filled.csv',index_col='Date')
cryptos_df = pd.read_csv('cryptos_data.csv', index_col='Date')

joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left') #mode - either do left with crypto and fill NA for stocks or do left on stocks and leave out some dates for cryptos
joined_df.index = pd.to_datetime(joined_df.index)

joined_df_weekly = joined_df.resample('W').last() #try aggregating on a weekly level
joined_df_3days = joined_df.resample('3D').last()# aggregating on a twice per week basis to arrive at the sweet spot of that 250 (1 year) timeseries length

In [ ]:
#Random Portfolios generation
tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

#Select top five sharpe ratio portfolios from a portfolio
sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)
top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [4]:
#RUN THE CLUSTERING WITH DIFFERENT SET UPS TO GET THE SILHOUETTE SCORES FOR COMPARISON

# n_clusters_list = [4,5,6,7]
# linkage_list=['single', 'average', 'complete']
# #window_sizes = [3,7,10,14,21,30,60]
# window_sizes = [21,30,60]


# def run_clustering_evaluation(df, window_sizes, method, moving_average=True, return_mode='arithmetic', df_input_name='DFWASNOTSPECIFIED'):

#     for w_size in window_sizes:

#         #return_mode = 'arithmetic'
#         #n_init = 3
#         #center = True
#         if moving_average:
#             df = df.rolling(window=w_size, center=True).mean()

#             smoothing = 'moving_average'
#         else:
#             smoothing = 'no_smoothing'

#         if len(df) < 150 and w_size > 30:
#             continue
#         silhouette_df = test_for_silhouette_score(df, n_clusters_list, method=method, return_mode=return_mode, n_init=3, linkage_list=linkage_list)

#         silhouette_df['return_mode'] = return_mode
#         silhouette_df['n_init'] = 3
#         silhouette_df['smoothing'] = smoothing
#         silhouette_df['window_size/span'] = w_size

#         silhouette_df.to_csv(f'silhouette_dfs/{method}_{smoothing}_{return_mode}_windowsize-{w_size}_{df_input_name}.csv')




# df_input_name = input('Put in the name of the df mode you are running for: ')
# for return_mode in ['arithmetic', 'geometric']:
#     run_clustering_evaluation(joined_df_3days, window_sizes, method='kmeans', moving_average=True, return_mode=return_mode, df_input_name=df_input_name)

In [5]:
#RUN FOR LABEL BALANCE

# method_loop = 'ahc'
# return_mode_loop = 'geometric'
# n_clus_loop = 7

# df_with_label_balance = pd.DataFrame()
# for df_dict in [{'weekly': joined_df_weekly}, {'3day': joined_df_3days}, {'full': joined_df}]:
#     for linkage in ['single', 'complete', 'average']:
#         for w in [3,7,10,14,21,30]:
#             output = label_balance(df_dict, w, method_loop, return_mode_loop, n_clus=n_clus_loop, linkage=linkage)
#             df_with_label_balance = pd.concat([df_with_label_balance, output])

# df_with_label_balance.reset_index(inplace=True)

#df_with_label_balance.to_csv(f'balance_datasets/balance_test_results_{method_loop}_{return_mode_loop}_{n_clus_loop}clusters.csv')

In [8]:
##############TEST###################
from functions import test_clustering_metrics
n_clusters_list = [4,5,6,7,9,12]
df_with_label_balance = pd.DataFrame()
for window in [3,7,10,14,30,60]:
    for df_dict in [{'weekly': joined_df_weekly}, {'3day': joined_df_3days}, {'full': joined_df}]:
        output = test_clustering_metrics(df_dict, n_clusters_list, method='kmeans', linkage_list=None,#['average', 'complete', 'single'], 
                               return_mode='geometric', window=window, n_init=3)
        df_with_label_balance = pd.concat([df_with_label_balance, output])

df_with_label_balance.to_csv('new_balance_silhouette/kmeans.csv', index=False)

In [ ]:
# def rejoin_dfs(folder_path, extract_cols_from_filename=False):
#     '''
#     extract_cols_from_filename is the option to handle legacy thing with the silhouette score tables when part of the information was stored in the filename
#     '''
#     dfs = pd.DataFrame()
#     for filename in os.listdir(folder_path):
#         file_path = os.path.join(folder_path, filename)  
#         df = pd.read_csv(file_path, index_col=False)
#         if extract_cols_from_filename:
#             df['filename'] = filename
#         dfs = pd.concat([dfs, df])
#     for col in ['index', 'Unnamed: 0']:
#         try:
#             dfs = dfs.drop(columns=[col])
#         except:
#             pass
#     if extract_cols_from_filename:
#         dfs['df_mode'] = dfs['filename'].apply(lambda x: x.split('-')[1].split('_')[1].replace('.csv', ''))
#         dfs['return_mode'] = dfs['filename'].apply(lambda x: x.split('_')[3])
#         dfs['linkage'] = dfs['linkage'].fillna('not_applicable')
    
#     dfs = dfs.drop_duplicates()

#     return dfs

In [35]:
kmeans_results = pd.read_csv('new_balance_silhouette/kmeans.csv')
kshape_results = pd.read_csv('new_balance_silhouette/kshape.csv')
ahc_results = pd.read_csv('new_balance_silhouette/ahc.csv')

results = pd.concat([kmeans_results, kshape_results, ahc_results]).drop(columns=['Unnamed: 0'])

In [46]:
from sklearn.preprocessing import MinMaxScaler
results[['silhouette_norm']] = MinMaxScaler().fit_transform(results[['silhouette_score']])
results['delta_norm'] = 1 - MinMaxScaler().fit_transform(results[['min_max_delta']])
results['total_score'] = (results['silhouette_norm'] + results['delta_norm']) / 2


results_filtered = results[results['silhouette_norm'] > 0.6]

best_configs = results_filtered.loc[results_filtered.groupby('method')['total_score'].idxmax()].sort_values(by='total_score', ascending=False)

In [47]:
best_configs

,clusters,silhouette_score,method,linkage,return_mode,window_size,df_mode,min_per_cluster,max_per_cluster,min_max_delta,silhouette_norm,delta_norm,total_score
10,9,0.173072,kshape,not_applicable,geometric,3,3day,0.0586,0.1802,0.1216,0.737209,0.940932,0.839071
96,4,0.189513,kmeans,not_applicable,geometric,60,3day,0.2027,0.3694,0.1667,0.761166,0.891599,0.826383
245,12,0.112789,ahc,complete,geometric,30,3day,0.0045,0.1818,0.1773,0.649365,0.880004,0.764685
96,4,0.258809,ahc,complete,geometric,7,full,0.0045,0.9409,0.9364,0.862143,0.049661,0.455902


In [38]:
results_filtered.sort_values(by='silhouette_score', ascending=False)

,clusters,silhouette_score,method,linkage,return_mode,window_size,df_mode,min_per_cluster,max_per_cluster,min_max_delta,silhouette_norm,delta_norm,total_score
210,4,0.353414,ahc,single,geometric,14,full,0.0045,0.9682,0.9636,1.000000,0.019908,0.509954
198,4,0.353414,ahc,average,geometric,14,full,0.0045,0.9682,0.9636,1.000000,0.019908,0.509954
90,4,0.350346,ahc,average,geometric,7,full,0.0045,0.9545,0.9500,0.995529,0.034785,0.515157
252,4,0.349257,ahc,average,geometric,30,full,0.0045,0.9682,0.9636,0.993942,0.019908,0.506925
264,4,0.349257,ahc,single,geometric,30,full,0.0045,0.9682,0.9636,0.993942,0.019908,0.506925
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,9,0.184258,kshape,not_applicable,geometric,3,full,0.0405,0.2432,0.2027,0.753508,0.852221,0.802864
13,5,0.184231,kshape,not_applicable,geometric,3,full,0.1306,0.3784,0.2477,0.753470,0.802997,0.778233
277,5,0.182295,ahc,complete,geometric,60,weekly,0.1000,0.4773,0.3773,0.750648,0.661234,0.705941
114,4,0.182000,ahc,complete,geometric,10,weekly,0.0091,0.5909,0.5818,0.750218,0.437541,0.593880


In [45]:
results.groupby('method')['total_score'].max()

method
ahc       0.764685
kmeans    0.826383
kshape    0.839071
Name: total_score, dtype: float64

In [19]:
###CRYPTOS DISTRIBUTION IN CLUSTERS:
# from collections import Counter
# _, tickers_with_labels, _, _ = run_clustering_model(joined_df, n_clus=3, model_name='kmeans', linkage='single', return_mode='arithmetic', n_init=3)

# cryptos_list = list(cryptos_df.columns) 
# crypto_clusters = {ticker: tickers_with_labels[ticker] for ticker in cryptos_list if ticker in tickers_with_labels}

# # Count how many cryptos are in each cluster
# distribution = Counter(crypto_clusters.values())